In [39]:
# imports
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import time

In [50]:
# loading table
FILE_PATH = '../winemag-data-130k-v2.csv'
DATA = pd.read_csv(FILE_PATH)

df = DATA[['country', 'province']]

df.head()

,country,province
0,Italy,Sicily & Sardinia
1,Portugal,Douro
2,US,Oregon
3,US,Michigan
4,US,Oregon


In [51]:
# dropping duplicated and reseting index
df = df.drop_duplicates().dropna()
df = df.reset_index().drop('index', axis=1)
df

,country,province
0,Italy,Sicily & Sardinia
1,Portugal,Douro
2,US,Oregon
3,US,Michigan
4,Spain,Northern Spain
...,...,...
420,Greece,Vin de Pays de Velvendo
421,Germany,Landwein Rhein
422,Greece,Lesbos
423,Portugal,Távora-Varosa


In [10]:
# scrapping google maps
# time sleep set to 15 to avoid google's captcha warning

import time

user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14'
headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}


lat_long_list = []
for idx, row in df.iterrows():
    URL = f'https://www.google.com/maps/search/{row.country}+{row.province}'
    r = requests.get(URL, headers=headers)
    soup = str(BeautifulSoup(r.content))
    try:
        lat_long = re.findall('@-?\d+\.\d+,-?\d+\.\d+', soup)              
        lat, long = re.split(',', lat_long[0])
        lat = re.sub('@', '', lat)
    except(IndexError, TypeError):
        lat = 0
        long = 0
#     print(soup)
    print(row.country, row.province, lat, long)
    lat_long_list.append([lat, long])
    time.sleep(15)
     



    



Italy Sicily & Sardinia 0 0
Portugal Douro 0 0
US Oregon 43.8041334 -120.5542012
US Michigan 44.3148443 -85.6023643
Spain Northern Spain 0 0
France Alsace 48.3181795 7.4416241
Germany Rheinhessen 0 0
US California 36.778261 -119.4179324
Germany Mosel 0 0
Argentina Other 0 0
Argentina Mendoza Province -32.8896247 -68.8526867
US Virginia 37.4315734 -78.6568942
France Beaujolais 46.1134349 4.7263863
Chile Colchagua Valley -34.6761419 -71.097317
Italy Southern Italy 0 0
Chile Maule Valley 0 0
France Bordeaux 44.837789 -0.57918
Chile Maipo Valley -33.7289899 -70.7778036
US Washington 47.7510741 -120.7401385
Italy Central Italy 0 0
France Champagne 48.9168895 4.1514325
France Burgundy 47.0525047 4.3837215
Australia South Australia -30.0002315 136.2091547
Portugal Tejo 39.7528442 -5.2532284
Chile Rapel Valley 0 0
Spain Galicia 42.5750554 -8.1338558
France France Other 46.227638 2.213749
Italy Tuscany 43.7710513 11.2486208
Austria Burgenland 47.1537165 16.2688797
US New York 40.7127753 -74.005

Romania Viile Timisului 0 0
US Illinois 40.6331249 -89.3985283
Turkey Aegean 0 0
South Africa Simonsberg-Paarl 0 0
Uruguay Juanico -34.5907059 -56.2566093
Bulgaria Black Sea Coastal 0 0
Brazil Santa Catarina -27.2423392 -50.2188556
South Africa Overberg -34.4897034 20.402376
Greece Atalanti Valley 0 0
Romania Sebes 45.9595588 23.5663758
Portugal Moscatel de Setúbal 0 0
Greece Macedonia 40.8176812 22.8527142
Portugal Ribatejo 0 0
Germany Germany 51.165691 10.451526
Portugal Palmela 38.569441 -8.9016937
Israel Galil 0 0
Slovenia Kras 0 0
Croatia Croatia 45.1 15.2000001
New Zealand Waipara -43.0551367 172.7590715
South Africa Olifants River -24.3688001 30.6473144
Uruguay Montevideo -34.9011127 -56.1645314
Brazil Campanha -21.8343864 -45.3990201
Israel Israel 31.046051 34.851612
Austria Neusiedlersee 0 0
Hungary Hungary 47.162494 19.5033041
Chile Lolol Valley 0 0
Chile Cauquenes Valley 0 0
South Africa Groenekloof -25.7740895 28.2204987
Portugal Alenquer 39.0577004 -9.0140033
Slovenia Gori

In [54]:
# creating Dataframe and check if the number of rows is consistent with number of rows of df
# 425 rows = 425 rows

df_lat_long = pd.DataFrame(lat_long_list, columns=['latitude', 'longtitude'])
df_lat_long

,latitude,longtitude
0,0,0
1,0,0
2,43.8041334,-120.5542012
3,44.3148443,-85.6023643
4,0,0
...,...,...
420,40.2545476,22.0742782
421,49.2313352,8.3860749
422,0,0
423,0,0


In [55]:
df

,country,province
0,Italy,Sicily & Sardinia
1,Portugal,Douro
2,US,Oregon
3,US,Michigan
4,Spain,Northern Spain
...,...,...
420,Greece,Vin de Pays de Velvendo
421,Germany,Landwein Rhein
422,Greece,Lesbos
423,Portugal,Távora-Varosa


In [56]:
# joining dataframes

provinces_lat_long = pd.concat([df, df_lat_long], axis=1)
provinces_lat_long

,country,province,latitude,longtitude
0,Italy,Sicily & Sardinia,0,0
1,Portugal,Douro,0,0
2,US,Oregon,43.8041334,-120.5542012
3,US,Michigan,44.3148443,-85.6023643
4,Spain,Northern Spain,0,0
...,...,...,...,...
420,Greece,Vin de Pays de Velvendo,40.2545476,22.0742782
421,Germany,Landwein Rhein,49.2313352,8.3860749
422,Greece,Lesbos,0,0
423,Portugal,Távora-Varosa,0,0


In [38]:
# checking the number of unrecognized provinces by the google maps
provinces_lat_long[provinces_lat_long['latitude'] == 0]

,country,province,latitude,longtitude
0,Italy,Sicily & Sardinia,0,0
1,Portugal,Douro,0,0
4,Spain,Northern Spain,0,0
6,Germany,Rheinhessen,0,0
8,Germany,Mosel,0,0
...,...,...,...,...
409,Greece,Thessalikos,0,0
411,China,China,0,0
414,Romania,Viile Timis,0,0
422,Greece,Lesbos,0,0


In [60]:
provinces_lat_long.to_csv('../provinces_latlong.csv', index=False, encoding='utf-8-sig')